# DNN project - Watermarks detection using YOLO

This notebook was ran on Kaggle, so the paths need to be updated

In [1]:
!pip install lightning -qqq
!pip install ultralytics -qqq
!pip install datasets -qqq
!pip install wget -qqq
!pip install torchmetrics -qqq
!pip install torchmetrics[detection] -qqq
!pip install pycocotool -qqq
!pip install faster-coco-eval -qqq

ERROR: Could not find a version that satisfies the requirement pycocotool (from versions: none)
ERROR: No matching distribution found for pycocotool


In [39]:
import lightning as pl
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision
from ultralytics import YOLO
from datasets import load_dataset
from torch.utils.data import DataLoader
from ultralytics.utils.benchmarks import benchmark
from torchmetrics.detection import MeanAveragePrecision
from pprint import pprint

## Dataset

In [3]:
%rm -rf /kaggle/working/data

In [4]:
%cd /kaggle/working

/kaggle/working


In [5]:
!pip install -qqq wget
import wget
import zipfile
import os

for i in ["train-yolo", "test-yolo", "val-yolo"]:
    wget.download(f"https://huggingface.co/datasets/bastienp/visible-watermark-pita/resolve/main/data/{i}.zip?download=true")

    with zipfile.ZipFile(f"{i}.zip", "r") as zip_ref:
        zip_ref.extractall("data")
    os.remove(f'{i}.zip')

In [6]:
# doesn't set correctly the path inside the file, better to import it by hand in Kaggle
wget.download(
            "https://huggingface.co/datasets/bastienp/visible-watermark-pita/resolve/main/data/metadata.yml?download=true",
            out="data/metadata.yml",
        )

'data/metadata.yml'

## Training with Ultralytics

In [7]:
DATA_PATH = "/kaggle/input/metadata"

In [9]:
yolo = YOLO('yolov8n.pt')
results = yolo.train(data=f'{DATA_PATH}/metadata.yml', epochs=15, imgsz=512, batch=128)

In [10]:
metrics = yolo.val()
metrics

### Testing

To test the model using Ultralytics, we need another metadata YAML file where the validition folder is set to the test folder.

In [11]:
TEST_DATA_PATH = "/kaggle/input/ultralytics-bench"
BEST_MODEL_PATH = "/kaggle/input/yolo-best/best.pt"

benchmark(model=BEST_MODEL_PATH, data=f'{TEST_DATA_PATH}/metadata_bench.yml', imgsz=512, device=0)

In [23]:
model = YOLO(BEST_MODEL_PATH)
TEST_PATH = "/kaggle/working/data/test-yolo"
results = model(f"{TEST_PATH}/images", stream=True, verbose=False)

preds = []
targets = []
for result in results:
    dict_res = dict(
        boxes=result.boxes.xywhn.cpu(),
        scores=result.boxes.conf.cpu(),
        labels=result.boxes.cls.cpu().int()
    )
    preds.append(dict_res)
    img_path = result.path
    img_id = img_path.split("/")[-1].split(".")[0]
    labels_path = f"{TEST_PATH}/labels/{img_id}.txt"
    with open(labels_path, "r") as f:
        lines = f.readlines()
        boxes = []
        classes = []
        for line in lines:
            box = list(map(float, line.strip().split(" ")))
            classes.append(int(box[0]))
            box = box[1:]
            boxes.append(box)
        dict_targets = {
            "boxes": torch.tensor(boxes),
            "labels": torch.tensor(classes)
        }
        targets.append(dict_targets)

In [24]:
pred = preds[0]
target = targets[0]
pred, target

({'boxes': tensor([[0.2073, 0.8510, 0.2894, 0.0658]]),
  'scores': tensor([0.8605]),
  'labels': tensor([1], dtype=torch.int32)},
 {'boxes': tensor([[0.2109, 0.8477, 0.3203, 0.0645]]), 'labels': tensor([1])})

In [27]:
metric = MeanAveragePrecision(iou_type="bbox", box_format="xywh", class_metrics=True)
metric.update(preds, targets)

In [28]:
pprint(metric.compute())

{'classes': tensor([0, 1], dtype=torch.int32),
 'map': tensor(0.8534),
 'map_50': tensor(0.9639),
 'map_75': tensor(0.9145),
 'map_large': tensor(-1.),
 'map_medium': tensor(-1.),
 'map_per_class': tensor([0.8511, 0.8556]),
 'map_small': tensor(0.8534),
 'mar_1': tensor(0.8739),
 'mar_10': tensor(0.8762),
 'mar_100': tensor(0.8762),
 'mar_100_per_class': tensor([0.8748, 0.8776]),
 'mar_large': tensor(-1.),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(0.8762)}
